In [30]:
import numpy as np
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [31]:
trainData = datasets.MNIST(root='data', train= True, download= True, transform= ToTensor())

In [32]:
testData = datasets.MNIST(root='data', train= False, download= True, transform= ToTensor())

In [33]:
print(f"{testData} \n\n {trainData}")

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor() 

 Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [35]:
trainLoader = torch.utils.data.DataLoader(trainData, batch_size= 64, shuffle= True)
testLoader = torch.utils.data.DataLoader(testData, batch_size= 100, shuffle=True)

In [36]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.convLayer1 = nn.Conv2d(in_channels= 1, out_channels= 10, kernel_size=5)
        self.convLayer2 = nn.Conv2d(in_channels= 10, out_channels= 20,kernel_size=5)
        self.conv2Drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.convLayer1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2Drop(self.convLayer2(x)), 2))
        x = x.view(-1,320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x,training=self.training)
        x = self.fc2(x)

        return F.log_softmax(x)

In [39]:

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

True